<a href="https://colab.research.google.com/github/dhruvbaliyan/Object-Detection/blob/main/Object_Detection_Through_SSD_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Beginner Warning**
This is dependent on a separate library which is not part of Tensorflow (but still made by the Tensorflow team).

Some beginners got "stuck" in the past when Tensorflow 2 came out, but this library was not compatible with Tensorflow 2.

Beginners often feel uncomfortable using older versions of libraries. This is not how things work in the real world.

See "Why bad programmers always need the latest version": https://youtu.be/BIXH_m6CT2I





In [1]:
!pip install -U "numpy==1.26.4" "tensorflow==2.19.0" "tf-models-official==2.19.1" "protobuf<=3.20.3"

In [2]:
!git clone https://github.com/tensorflow/models.git


Cloning into 'models'...
remote: Enumerating objects: 102910, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 102910 (delta 164), reused 101 (delta 93), pack-reused 102687 (from 3)
Receiving objects: 100% (102910/102910), 642.87 MiB | 20.87 MiB/s, done.
Resolving deltas: 100% (74333/74333), done.


In [3]:
%cd /content/models/research

/content/models/research


In [4]:
!ls

adversarial_text    deeplab		   marco	     seq_flow_lite
attention_ocr	    deep_speech		   nst_blogpost      slim
audioset	    delf		   object_detection  vid2depth
autoaugment	    efficient-hrl	   pcl_rl
cognitive_planning  lfads		   README.md
cvt_text	    lstm_object_detection  rebar


##### **TensorFlow** **Model** **Garden**

A big collection of implemented

*   ML models (vision, NLP, detection, etc.) built with TensorFlow.

*   Contains reference implementations of SOTA papers, plus training scripts, configs, and example datasets.

*   Meant to show best practices and give you ready-to-run code you can fine-tune on your own data.



`Ready-made AI models you can use right away`

**SOTA- State Of The Art**

✔ It is the best-performing model currently known

✔ On a specific task

✔ Compared using a benchmark or public dataset



####**Install Protobuf**

No need to do this if you are on Colab.

If you're doing this locally, download the latest Protobuf library for your OS from https://github.com/google/protobuf/releases

The filename should look like "protoc-*-*.zip".

Example: protoc-3.17.3-win64.zip if you are using 64-bit Windows.

Assuming you've unzipped this zip file to \<path>, the next step is to add \<path>/bin to your PATH environment variable (on Linux or Mac).

Once Protobuf has been successfully installed, you can run the following command (note: must be done from the models/research folder).

In [5]:
!apt-get install -y protobuf-compiler

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.4).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [6]:
!protoc object_detection/protos/*.proto --python_out=.

| Part                              | Meaning                                                              |
| --------------------------------- | -------------------------------------------------------------------- |
| `!`                               | "Run a shell command" (used in Jupyter Notebook / Colab)             |
| `cd models/research/`             | Move into the `research` folder where the Object Detection API lives |
| `&&`                              | Run the next command only if cd worked                               |
| `protoc`                          | Protobuf compiler tool                                               |
| `object_detection/protos/*.proto` | Compile **all** .proto files inside the `protos` folder              |
| `--python_out=.`                  | Output the generated Python files **into the same folder**           |


#### **Install the object detection API**

In [7]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow-model-optimization to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tensorflow-model-optimization to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking 

In [10]:
import os
import pathlib
import time
import tensorflow as tf
import imageio

import numpy as np
import matplotlib.pyplot as plt

from object_detection.utils import \
  label_map_util, visualization_utils as viz_utils

from PIL import Image
%matplotlib inline


In [11]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

#### **Download Video**

In [18]:
%cd /content/

/content


In [20]:
!wget -nc https://lazyprogrammer.me/course_files/cnn_class2_videos.zip

--2025-12-07 07:09:05--  https://lazyprogrammer.me/course_files/cnn_class2_videos.zip
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2073140 (2.0M) [application/zip]
Saving to: ‘cnn_class2_videos.zip’

cnn_class2_videos.z 100%[===================>]   1.98M  --.-KB/s    in 0.05s   

2025-12-07 07:09:05 (40.7 MB/s) - ‘cnn_class2_videos.zip’ saved [2073140/2073140]



In [21]:
!unzip cnn_class2_videos.zip

Archive:  cnn_class2_videos.zip
  inflating: catdog.mp4              
  inflating: safari.mp4              
  inflating: traffic.mp4             


In [22]:
!ls

catdog.mp4  cnn_class2_videos.zip  models  safari.mp4  sample_data  traffic.mp4


In [23]:
INPUT_VIDEO = ['catdog.mp4', 'safari.mp4', 'traffic.mp4']

#### **Download and extract model files**

Get URLs from the "Object Detection Zoo": https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [45]:
url= 'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d2_coco17_tpu-32.tar.gz'

In [46]:
PATH_TO_MODEL_DIR = tf.keras.utils.get_file(
    fname= 'efficientdet_d2_coco17_tpu-32',
    origin=url,
    untar=True
)
PATH_TO_MODEL_DIR

'/root/.keras/datasets/efficientdet_d2_coco17_tpu-32'

In [73]:
!ls -F {PATH_TO_MODEL_DIR}

ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/


#### **Download Labels File**

Label files can be found here: https://github.com/tensorflow/models/tree/master/research/object_detection/data

You probably won't need these since Object Detection Zoo contains only models trained on COCO.



In [47]:
url1 = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'

In [55]:
!rm {PATH_TO_LABELS}
!wget -O {PATH_TO_LABELS} {url1}

--2025-12-07 07:25:11--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5056 (4.9K) [text/plain]
Saving to: ‘/root/.keras/datasets/mscoco_label_map.pbtxt’

/root/.keras/datase 100%[===================>]   4.94K  --.-KB/s    in 0s      

2025-12-07 07:25:11 (54.8 MB/s) - ‘/root/.keras/datasets/mscoco_label_map.pbtxt’ saved [5056/5056]



In [58]:
PATH_TO_LABELS

'/root/.keras/datasets/mscoco_label_map.pbtxt'

In [59]:
!head {PATH_TO_LABEL}

item {
  name: "/m/01g317"
  id: 1
  display_name: "person"
}
item {
  name: "/m/0199g"
  id: 2
  display_name: "bicycle"
}


#### **Load in the Label Map**

🧠 What’s happening when you run the detector

* Your downloaded model (e.g. ssd_resnet101_v1_fpn_640x640_coco17_tpu-8)
was trained on COCO dataset.

* During inference, the model outputs things like:


```
detections['detection_boxes']      # bounding boxes
detections['detection_classes']    # class IDs (1, 2, 3, ...)
detections['detection_scores']     # confidence scores

```



#### **Load in the model**

In [74]:
PATH_TO_SAVED_MODEL = os.path.join(PATH_TO_MODEL_DIR, 'ssd_resnet101_v1_fpn_640x640_coco17_tpu-8', 'saved_model')

In [75]:
!ls {PATH_TO_SAVED_MODEL}

assets	saved_model.pb	variables


In [76]:
print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 83.99462151527405 seconds


#### **Load in the Label**

In [77]:
category_index = label_map_util.create_category_index_from_labelmap(
    PATH_TO_LABELS,
    use_display_name=True
)

In [78]:
category_index

{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suitc

#### **Object Detection**

To make the code snippet you provided suitable for a text cell in a Colab notebook, you should convert the Python function into a clear explanation of its steps, dependencies, and purpose.

Here is a structured explanation you can use in a Colab text cell:

-----

# 🤖 Object Detection Function Explained

This text cell provides a detailed explanation of the Python function `detectObject(image_np)`, which is typically used in a Jupyter or Colab environment for real-time or batch object detection using a pre-trained TensorFlow model.

The function takes a NumPy image array, runs it through a detection model, and draws the resulting bounding boxes and labels onto the image.

## 📦 Prerequisites (Assumed Imports)

The function relies on several external libraries and variables that must be defined and imported in the code cells:

  * `tf`: **TensorFlow** (for Tensor operations and the model).
  * `np`: **NumPy** (for array manipulation).
  * `detect_fn`: The **loaded TensorFlow detection model** (a function that takes an image tensor and returns detection results).
  * `viz_utils`: The **visualization utilities** (often from `object_detection.utils.visualization_utils`).
  * `category_index`: A **dictionary** mapping numerical class IDs to human-readable labels (e.g., `{1: {'id': 1, 'name': 'person'}}`).

-----

## 🔬 Function Breakdown: `detectObject(image_np)`

| Step | Code | Purpose |
| :--- | :--- | :--- |
| **1. Preprocessing (Input)** | `input_tensor = tf.convert_to_tensor(image_np)`<br>`input_tensor = input_tensor[tf.newaxis, ...]` | Converts the input **NumPy array** (`image_np`) into a **TensorFlow Tensor**. A new axis (`tf.newaxis`) is added to the front to create the **batch dimension** (required input shape: `[1, H, W, C]`). |
| **2. Inference** | `detections = detect_fn(input_tensor)` | Passes the prepared tensor to the pre-loaded model (`detect_fn`) to perform object detection. The model returns a dictionary of raw detection outputs (boxes, classes, scores). |
| **3. Postprocessing (Output)** | `num_detections = int(detections.pop('num_detections'))`<br>`detections = {...}`<br>`detections['detection_classes'] = ...astype(np.int64)` | Extracts the actual number of detections and converts the raw detection Tensors (which include the batch dimension) into **NumPy arrays** containing only the valid results. Ensures class IDs are integers. |
| **4. Visualization** | `image_np_with_detections = image_np.copy()`<br>`viz_utils.visualize_boxes_and_labels_on_image_array(...)` | Creates a copy of the original image. The `visualize_boxes_and_labels_on_image_array` function draws **bounding boxes**, **scores**, and **labels** onto this image copy based on the processed detection data.<br>*Note: Only detections with a `min_score_thresh` of **0.30** or higher are drawn.* |
| **5. Return Value** | `return image_np_with_detections` | Returns the modified image (as a NumPy array) with the detection results overlaid. |

-----

## 🛠️ Typical Use Case

This function is commonly placed inside a loop to process video streams or multiple images, creating a pipeline for real-time object detection applications.

In [79]:
def detectObject(image_np):
  # Numpy ----> Tensor
  input_tensor = tf.convert_to_tensor(image_np)
  # New Axis
  input_tensor = input_tensor[tf.newaxis, ...]

  detections = detect_fn(input_tensor)

  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
  detections['num_detections'] = num_detections

  # detection_classes should be ints.
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  image_np_with_detections = image_np.copy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.30,
        agnostic_mode=False)

  return image_np_with_detections

In [84]:
def object_in_video(input_video):
  print(f'Running for {input_video} ...', end='')
  video_reader = imageio.get_reader(f'{input_video}')
  video_writer = imageio.get_writer(f'{input_video.replace(".mp4", "")}_annotated.mp4', fps=10)

  # loop through and process each frame
  t0 = time.time()
  n_frames = 0
  for frame in video_reader:
      n_frames += 1
      new_frame = detectObject(frame)

      # instead of plotting image, we write the frame to video
      video_writer.append_data(new_frame)

  fps = n_frames / (time.time() - t0)
  print("Frames processed: %s, Speed: %s fps" % (n_frames, fps))

  # clean up
  video_writer.close()

In [85]:
object_in_video(INPUT_VIDEO[0])

Running for catdog.mp4 ...Frames processed: 50, Speed: 0.15847605514254753 fps


In [86]:
object_in_video(INPUT_VIDEO[1])

Running for safari.mp4 ...

Frames processed: 100, Speed: 0.16740313619177258 fps


In [87]:
object_in_video(INPUT_VIDEO[2])

Running for traffic.mp4 ...Frames processed: 70, Speed: 0.1674075409231666 fps


In [90]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
!cp "/content/catdog_annotated.mp4"  "/content/drive/MyDrive/"
